In [1]:
%matplotlib notebook
from osgeo import gdal,gdalconst
import matplotlib.pyplot as plt
import numpy as np
import os
import geopandas as gpd
import pandas as pd
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap
from sys import argv

/home/gbessardon/.local/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
def create_folder(path):
    
    exists =  os.path.exists(path) #check if the folder exists
    
    if exists == False:
        
        os.mkdir(path) #create the folder if it doesnt already exist

In [3]:
def get_data(fname):

    gdal.UseExceptions()
    ds = gdal.Open(fname)
    data = ds.ReadAsArray()
    gt = ds.GetGeoTransform()
    #
    #
    #
    xres = gt[1]
    yres = gt[5]
    #
    xmin = gt[0]
    ymin = gt[3]
    #
    xmax = gt[0] + (xres * ds.RasterXSize)
    ymax = gt[3] + (yres * ds.RasterYSize)
#    else:
#        xmax = gt[0] + (xres * ds.RasterXSize) 
#        ymax = gt[3] + (yres * ds.RasterYSize) 


    X=np.arange(xmin+xres,xmax+xres,xres)
    Y=np.arange(ymin+yres,ymax+yres,yres)

    return (X,Y,data,ds)

In [4]:
def Create_copernicus_corect_projection(fn,fnecosg,dst_filename = 'temp_ecosg.tiff'):
    # create a temporary file with the ECO-SG file projection
    #Open the Copernicus tiff file
    src=gdal.Open(fn)
    src_proj = src.GetProjection()
    src_geotrans = src.GetGeoTransform()
    xmax = src_geotrans[0] + (src_geotrans[1] * src.RasterXSize)
    ymax = src_geotrans[3] + (src_geotrans[5] * src.RasterYSize)

    #Open the ecosg file to get the projection details
    match_filename = fnecosg
    match_ds = gdal.Open(match_filename)
    match_proj = match_ds.GetProjection()
    match_geotrans = match_ds.GetGeoTransform()
    match_geotrans=(src_geotrans[0],match_geotrans[1],match_geotrans[2],src_geotrans[3],match_geotrans[4],match_geotrans[5])
    wide = int((xmax-match_geotrans[0])/match_geotrans[1])
    high = int((ymax-match_geotrans[3])/match_geotrans[5])

    #Create the Copernicus tiff file with the right projection
    #dst_filename = 'temp_ecosg.tiff'
    dst = gdal.GetDriverByName('GTiff').Create(dst_filename, wide, high, 1, gdal.GDT_Byte)
    dst.SetGeoTransform( match_geotrans )
    dst.SetProjection( match_proj)

    # Do the work
    gdal.ReprojectImage(src, dst, src_proj, match_proj, gdalconst.GRA_Bilinear)

    del dst
    return(dst_filename)

In [5]:
def overwriteECOSG_data(fnecosg,outFileName,fn2020):
    #Open ECOCLIMAP_SG climatology data
    (Xc,Yc,datac,dsc)=get_data(fnecosg)
    [cols, rows] = datac.shape
    # Overwrite ECO_SG data using the temp file
    (X,Y,data2020,ds2020)=get_data(fn2020)
    #data2020[data2020==255]=0 #no data
    D2020=data2020/3 #scaling to ecosg scaling (Copernicus scaling 1/30 ecosg scaling 1/10)
    gt2020=ds2020.GetGeoTransform()
    idxmin=np.argmin(np.abs(Xc-gt2020[0]))
    idymin=np.argmin(np.abs(Yc-gt2020[3]))
    idxmax=idxmin+ds2020.RasterXSize
    idymax=idymin+ds2020.RasterYSize
    datac[idymin:idymax,idxmin:idxmax]=D2020
    arr_out=datac
    # Save the new data in the outputfile
    outdata = dsc.GetDriver().Create(outFileName, rows, cols, 1, gdal.GDT_Byte)
    outdata.SetGeoTransform(dsc.GetGeoTransform())##sets same geotransform as input
    outdata.SetProjection(dsc.GetProjection())##sets same projection as input
    outdata.GetRasterBand(1).WriteArray(arr_out)
    #outdata.GetRasterBand(1).SetNoDataValue(10000)##if you want these values transparent
    outdata.FlushCache() ##saves to disk!!
    outdata = None
    band=None
    dsc=None
    #Remove the temporay file
    os.remove(fn2020)
    #Array=gdal.Open(outFileName).ReadAsArray()
    return

In [6]:
def alltogether(fn,fnecosg,outFileName):
    fn2020=Create_copernicus_corect_projection(fn,fnecosg)
    overwriteECOSG_data(fnecosg,outFileName,fn2020)

In [7]:
#fn='/data/gbessardon/LAI_2014/20140331/c_gls_LAI300-LAI_201403310000_CUSTOM_PROBAV_V1.0.1.tiff' # Copernicus tiff data
#fnecosg='/data/gbessardon/openECOCLIMAP_SG/300M/LAI_0125_c.dir_2' #Uncompressed ECOSG climatology file
#outFileName='/data/gbessardon/LAI_2014/LAI_0125_c.dir_2' # output filename
#alltogether(fn,fnecosg,outFileName)

In [8]:
dire='/data/gbessardon/LAI_2018/'# yearly Copernicus folder
year=dire.split('LAI_')[1][0:4] # depends on your directory path 
direm=[os.path.join(dire,d) for d in np.sort(os.listdir(dire)) if d.startswith(year)]# monthly Copernicus folder
f10d=np.sort([os.path.join(direm,dm,f) for dm in direm for f in os.listdir(dm) if f.startswith('c_gls_LAI-RT6-LAI_')])
outputdir=os.path.join(dire,'LAI_2020_to_hdr_outputs')
create_folder(outputdir)# Createoutputdir

In [9]:
direcosg='/data/gbessardon/openECOCLIMAP_SG/300M/'
FNecosg=np.sort([os.path.join(direcosg,f) for f in os.listdir(direcosg) if f.endswith('c.dir_2')])

In [10]:
for i,fnecosg in enumerate(FNecosg):
    outFileName=os.path.join(outputdir,fnecosg.replace(direcosg,''))
    fn=f10d[i]
    alltogether(fn,fnecosg,outFileName)
    print(outFileName)
    print('....Created')

/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0105_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0115_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0125_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0205_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0215_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0225_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0305_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0315_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0325_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0405_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0415_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LAI_0425_c.dir_2
....Created
/data/gbessardon/LAI_2018/LAI_2020_to_hdr_outputs/LA